## Openscale Verizon Wrapper function deploy

In [ ]:
# Environment variables. Edit these with correct values.

SCORING_URL = ''
FUNCTION_NAME = ''
DEPLOYMENT_NAME = ''
DEPLOYMENT_SPACE = ''
SUBSCRIPTION_NAME = ''
TARGET = ''

In [1]:
## import libraries
import json
!pip install --proxy=proxy.ebiz.verizon.com:80 ibm-ai-openscale | tail -n 1
    
from watson_machine_learning_client import WatsonMachineLearningAPIClient

wml_credentials = {"username":"admin", "password":"password", "url":"<EDIT THIS>", "instance_id":"wml_local", "version":"2.5.0"}
wml_client = WatsonMachineLearningAPIClient(wml_credentials)

for space in wml_client.spaces.get_details().get('resources'):
    if space.get('entity').get('name')== DEPLOYMENT_SPACE:
        wml_client.set.default_space(space.get('metadata').get('guid'))
        print('Success!')

Success!


In [8]:
def deployable_function():
    
    def score(payload):
        import requests
        
        fields = payload.get('input_data')[0].get('fields')

        def convert_payload(payload, fields):
            fields = fields

            rows = payload.get('input_data')[0].get('values')
            rows = [list(map(int,row)) for row in rows]
            
            payload_list = []
            
            for values in rows:
                value =  {"serviceHeader": {"clientId": "Streams","clientTransactionId": "<Unique Random Number>", "modelName": "channel-hop-model","modelVersion": "v1", "custId": "Not Applicable",  "acctNo": "Not Applicable", "mtn": "9999999999"  }, 
             "serviceRequest": { "arguments": {  "input_type": "dataframe","input": dict(zip(fields, values))}}}
                payload_list.append(value)
                
            return payload_list
        
        payloads = convert_payload(payload, fields)
        
        response_list = []
        scoring_url = SCORING_URL

        for payload in payloads:
            response = requests.post(scoring_url, json=payload, verify = False).json()
            values = [response.get('serviceResponse').get('result').get('prediction')[0]  , [v for k,v in response.get('serviceResponse').get('result').get('probabilities').items() ] ]
                       
            response_list.append(values)
                      
        return {'predictions': [{'fields': ['prediction', 'probability'], 'values': response_list }]}
    return score

In [24]:
scoring_payload = {"input_data":[{"fields": list(request.get('serviceRequest').get('arguments').get('input').keys()), "values":[list(map(str,list(request.get('serviceRequest').get('arguments').get('input').values())))  ]  }]}

deployable_function()(scoring_payload)

{'predictions': [{'fields': ['prediction', 'probability'],
   'values': [['ch_other', [0.02602, 0.8867, 0.06614, 0.00605, 0.01509]]]}]}

In [12]:

for function in wml_client.repository.get_details().get('functions').get('resources'):
    if function.get('entity').get('name')== FUNCTION_NAME:
        function_uid = function.get('metadata').get('guid')
        wml_client.repository.delete(function_uid)
        print('deleting function '+ function_uid)
               
meta_data = { wml_client.repository.FunctionMetaNames.NAME : FUNCTION_NAME }
function_details = wml_client.repository.store_function( meta_props=meta_data, function=deployable_function )

Using default runtime with uid: ai-function_0.1-py3.6


In [13]:
function_uid = function_details.get('metadata').get('guid')

In [14]:
meta_props = {
wml_client.deployments.ConfigurationMetaNames.NAME: DEPLOYMENT_NAME,
wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
}
deployment_details = wml_client.deployments.create(function_uid, meta_props)



#######################################################################################

Synchronous deployment creation for uid: '51fdf1e1-79e5-4e3d-a0cd-5ebb75d98d96' started

#######################################################################################


initializing....
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='1e4e982f-e80a-4331-a184-f0e1678f7cc6'
------------------------------------------------------------------------------------------------




In [17]:
import time
import requests

scoring_url = deployment_details.get('entity').get('status').get('online_url').get('url')
ml_token = wml_client.wml_token
headers = {"Content-Type":"application/json", "Authorization": "Bearer " + ml_token}
start_time= time.time()

response = requests.post(scoring_url, json = scoring_payload, headers=headers, verify=False)
response_time = int((time.time()-start_time)*1000)
response_data = response.json()

In [32]:
response_data

{'predictions': [{'fields': ['prediction', 'probability'],
   'values': [['ch_other', [0.02602, 0.8867, 0.06614, 0.00605, 0.01509]],
    ['ch_other', [0.02602, 0.8867, 0.06614, 0.00605, 0.01509]]]}]}

In [33]:
#!pip install --proxy=proxy.ebiz.verizon.com:80 ibm_ai_openscale | tail -n 1
from ibm_ai_openscale import APIClient4ICP
from ibm_ai_openscale.supporting_classes import PayloadRecord

aios_credentials = {
                      
                      "username": "admin",
                      "password": "password",
                      # address should be replaced with ip, port pair to be used in scripts outside ICP
                      "url": wml_credentials.get('url')
                   }

ai_client = APIClient4ICP(aios_credentials)
for subscription in ai_client.data_mart.subscriptions.get_details().get('subscriptions'):
    if subscription.get('entity').get('asset').get('name')== SUBSCRIPTION_NAME:
        subscription  = ai_client.data_mart.subscriptions.get(subscription.get('metadata').get('guid'))
        print('subscription')

subscription


This code removes previous subscriptions to the model to refresh the monitors with the new model and new data.

In [ ]:
subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
print(subscriptions_uids)
for subscription in subscriptions_uids:
    sub_name = ai_client.data_mart.subscriptions.get_details(subscription)['entity']['asset']['name']
    if sub_name == DEPLOYMENT_NAME:
        ai_client.data_mart.subscriptions.delete(subscription)
        print('Deleted existing subscription for', DEPLOYMENT_NAME)

This code creates the model subscription in OpenScale using the Python client API. Note that we need to provide the model unique identifier, and some information about the model itself.

In [ ]:
subscription = ai_client.data_mart.subscriptions.add(WatsonMachineLearningAsset(
    model_uid,
    problem_type=ProblemType.BINARY_CLASSIFICATION, # Change depending on problem type
    input_data_type=InputDataType.STRUCTURED,       # Change depending on problem type
    label_column= TARGET,
    transaction_id_column='transaction_id', # Optional
    prediction_column='prediction',
    probability_column='probability',
    # Replace values below with your values. Note: this step can be also done through the UI.
    feature_columns = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"],
    categorical_columns = ["CheckingStatus","CreditHistory","LoanPurpose","ExistingSavings","EmploymentDuration","Sex","OthersOnLoan","OwnsProperty","InstallmentPlans","Housing","Job","Telephone","ForeignWorker"],
    training_data_reference = {'connection': {'database_name': 'BLUDB',
                                              'hostname': 'dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net',
                                              'password': 'khhz72v+6mcwwkfv',
                                              'username': 'cmb91569'},
                                             'location': {'schema_name': 'CMB91569',
                                              'table_name': 'CREDIT_RISK_TRAIN_DATA'},
                                             'type': 'db2'}
    
))

if subscription is None:
    print('Subscription already exists; get the existing one')
    subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
    for sub in subscriptions_uids:
        if ai_client.data_mart.subscriptions.get_details(sub)['entity']['asset']['name'] == DEPLOYMENT_NAME:
            subscription = ai_client.data_mart.subscriptions.get(sub)

In [35]:
request_data = scoring_payload.get('input_data')[0]
response_data = response_data

records = [PayloadRecord(request=request_data, response=response_data, response_time=response_time), 
                PayloadRecord(request=request_data, response=response_data, response_time=response_time)]

subscription.payload_logging.store(records=records)

In [39]:
subscription.payload_logging.show_table()

b60e4ed9-6c2b-4489-b446-49cf11425189-1,2020-07-24 13:39:08.493000+00:00,1e4e982f-e80a-4331-a184-f0e1678f7cc6,None,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,1,0,1,1,1,0,0,1,0,0,1,1,0,0,0,0,0,0,1,ch_other,"[0.02602, 0.8867, 0.06614, 0.00605, 0.01509]",None
aee77b50-8fe0-4482-bed1-13ff453c8c68-1,2020-07-24 13:39:08.493000+00:00,1e4e982f-e80a-4331-a184-f0e1678f7cc6,None,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,1,0,1,1,1,0,0,1,0,0,1,1,0,0,0,0,0,0,1,ch_other,"[0.02602, 0.8867, 0.06614, 0.00605, 0.01509]",None


In [18]:
import pandas as pd
from project_lib import Project
project = Project.access()
project.get_files()

df = pd.read_csv('/project_data/data_asset/finalData.csv', nrows = 20000, index_col='rec_no')            

In [28]:
subscription.quality_monitoring.run(background_mode=False)




 Waiting for end of quality monitoring run 8040ce22-475b-4ca9-a1da-4b15a37d2eca 




initializing
running..
completed

---------------------------
 Successfully finished run 
---------------------------




{'stages': [{'name': 'Prerequisite Check',
   'completed_at': '2020-07-20T19:18:20.064Z',
   'started_at': '2020-07-20T19:18:19.051Z',
   'id': 1,
   'properties': {'training_columns': ['chdig0var1',
     'chdig0var2',
     'chdig0var3',
     'chdig0var4',
     'chdig0var5',
     'chdig0var6',
     'chdig0var7',
     'chdig0var8',
     'chdig0var9',
     'chdig0var10',
     'chdig0var11',
     'chdig0var12',
     'chdig0var13',
     'chdig0var14',
     'chdig72var1',
     'chdig72var2',
     'chdig72var3',
     'chdig72var4',
     'chdig72var5',
     'chdig72var6',
     'chdig72var7',
     'chdig72var8',
     'chdig72var9',
     'chdig72var10',
     'chdig72var11',
     'chdig72var12',
     'chdig72var13',
     'chdig72var14',
     'chdig14var1',
     'chdig14var2',
     'chdig14var3',
     'chdig14var4',
     'chdig14var5',
     'chdig14var6',
     'chdig14var7',
     'chdig14var8',
     'chdig14var10',
     'chdig14var11',
     'chdig14var12',
     'chdig14var13',
     'chdig14var14'

In [9]:
# Sample request from original endpoint

request = {
"serviceHeader": {
"clientId": "Streams",
"clientTransactionId": "<Unique Random Number>",
"modelName": "channel-hop-model",
        "modelVersion": "v1",
"custId": "Not Applicable",
"acctNo": "Not Applicable",
"mtn": "9999999999"
},
"serviceRequest": {
"arguments": {
"input_type": "dataframe",
"input": {
"chdig0var1": 1,
"chdig0var2": 0,
"chdig0var3": 0,
"chdig0var4": 0,
"chdig0var5": 0,
"chdig0var6": 0,
"chdig0var7": 0,
"chdig0var8": 0,
"chdig0var9": 0,
"chdig0var10": 0,
"chdig0var11": 0,
"chdig0var12": 0,
"chdig0var13": 0,
"chdig0var14": 0,
"chdig72var1": 0,
"chdig72var2": 0,
"chdig72var3": 0,
"chdig72var4": 0,
"chdig72var5": 0,
"chdig72var6": 0,
"chdig72var7": 0,
"chdig72var8": 0,
"chdig72var9": 0,
"chdig72var10": 0,
"chdig72var11": 0,
"chdig72var12": 0,
"chdig72var13": 0,
"chdig72var14": 0,
"chdig14var1": 0,
"chdig14var2": 0,
"chdig14var3": 0,
"chdig14var4": 0,
"chdig14var5": 0,
"chdig14var6": 0,
"chdig14var7": 0,
"chdig14var8": 0,
"chdig14var10": 0,
"chdig14var11": 0,
"chdig14var12": 0,
"chdig14var13": 0,
"chdig14var14": 0,
"chtxtvar1": 1,
"chtxtvar3": 1,
"chtxtvar4": 1,
"chtxtvar5": 0,
"chtxtvar6": 1,
"chtxtvar7": 0,
"chtxtvar8": 0,
"chtxtvar9": 0,
"chtxtvar10": 0,
"chtxtvar11": 0,
"chtxtvar12": 0,
"chtxtvar13": 1,
"chtxtvar14": 0,
"chtxtvar15": 0,
"chtxtvar16": 0,
"chtxtvar17": 1,
"chtxtvar18": 0,
"chtxtvar19": 0,
"chtxtvar20": 0,
"chtxtvar21": 0,
"chtxtvar22": 0,
"chtxtvar23": 0,
"chtxtvar25": 0,
"chtxtvar26": 0,
"chtxtvar27": 0,
"chtxtvar29": 0,
"chtxtvar30": 0,
"chtxtvar31": 0,
"chtxtvar32": 0,
"chtxtvar33": 0,
"chtxtvar34": 0,
"chtxtvar35": 0,
"chtxtvar36": 0,
"chtxtvar37": 0,
"chtxtvar38": 0,
"chtxtvar39": 0,
"chtxtvar40": 0,
"chtxtvar41": 0,
"chtxtvar42": 0,
"chtxtvar43": 0,
"chtxtvar44": 0,
"chtxtvar45": 0,
"chtxtvar46": 0,
"chtxtvar47": 1,
"chtxtvar48": 0,
"chtxtvar49": 0,
"chtxtvar50": 0,
"chtxtvar51": 0,
"chtxtvar52": 0,
"chtxtvar53": 0,
"chtxtvar55": 0,
"chtxtvar56": 0,
"chtxtvar57": 0,
"chtxtvar58": 0,
"chtxtvar59": 0,
"chtxtvar60": 1,
"chtxtvar61": 0,
"chtxtvar62": 0,
"chtxtvar63": 0,
"chtxtvar64": 1,
"chtxtvar65": 0,
"chtxtvar66": 0,
"chtxtvar67": 0,
"chtxtvar68": 0,
"chtxtvar69": 0,
"chtxtvar70": 0,
"chtxtvar71": 0,
"chtxtvar72": 0,
"chtxtvar73": 0,
"chtxtvar74": 0,
"chtxtvar75": 0,
"chtxtvar76": 0,
"chtxtvar77": 0,
"chtxtvar78": 0,
"chtxtvar80": 0,
"chtxtvar81": 0,
"chtxtvar82": 0,
"chtxtvar84": 0,
"chtxtvar85": 0,
"chtxtvar86": 1,
"chtxtvar87": 0,
"chtxtvar88": 0,
"chtxtvar89": 0,
"chtxtvar90": 0,
"chtxtvar91": 0,
"chtxtvar92": 0,
"chtxtvar93": 1,
"chtxtvar94": 0,
"chtxtvar95": 0,
"chtxtvar96": 1,
"chtxtvar97": 0,
"chtxtvar98": 0,
"chtxtvar99": 0,
"chtxtvar100": 0,
"chtxtvar101": 0,
"chtxtvar102": 0,
"chtxtvar103": 0,
"chtxtvar105": 0,
"chtxtvar106": 0,
"chtxtvar107": 0,
"chtxtvar108": 0,
"chtxtvar109": 0,
"chtxtvar110": 0,
"chtxtvar111": 0,
"chtxtvar112": 0,
"chtxtvar113": 0,
"chtxtvar114": 0,
"chtxtvar115": 0,
"chtxtvar116": 0,
"chtxtvar118": 0,
"chtxtvar119": 0,
"chtxtvar120": 0,
"chtxtvar122": 0,
"chtxtvar123": 0,
"chtxtvar124": 0,
"chtxtvar125": 0,
"chtxtvar126": 0,
"chtxtvar127": 0,
"chtxtvar128": 0,
"chtxtvar129": 0,
"chtxtvar130": 0,
"chtxtvar131": 0,
"chtxtvar132": 0,
"chtxtvar133": 0,
"chtxtvar135": 0,
"chtxtvar136": 0,
"chtxtvar137": 0,
"chtxtvar138": 0,
"chtxtvar139": 0,
"chtxtvar140": 0,
"chtxtvar142": 0,
"chtxtvar143": 0,
"chtxtvar144": 0,
"chtxtvar146": 0,
"chtxtvar147": 0,
"chtxtvar148": 0,
"chtxtvar149": 0,
"chtxtvar150": 0,
"chtxtvar151": 0,
"chtxtvar153": 0,
"chtxtvar154": 0,
"chtxtvar156": 0,
"chtxtvar157": 0,
"chtxtvar158": 0,
"chtxtvar159": 0,
"chtxtvar160": 0,
"chtxtvar161": 0,
"chtxtvar162": 0,
"chtxtvar163": 0,
"chtxtvar165": 0,
"chtxtvar167": 0,
"chtxtvar168": 0,
"chtxtvar170": 0,
"chtxtvar172": 0,
"chtxtvar174": 0,
"chtxtvar175": 0,
"chtxtvar176": 0,
"chtxtvar177": 0,
"chtxtvar180": 0,
"chtxtvar181": 0,
"chtxtvar182": 0,
"chtxtvar184": 0,
"chtxtvar186": 0,
"chtxtvar188": 0,
"chtxtvar189": 0,
"chtxtvar190": 0,
"chtxtvar191": 0,
"chtxtvar192": 0,
"chtxtvar194": 0,
"chtxtvar195": 0,
"chtxtvar196": 0,
"chtxtvar197": 0,
"chtxtvar198": 0,
"chtxtvar199": 0,
"chtxtvar200": 0,
"chtxtvar201": 0,
"chtxtvar202": 1,
"chtxtvar203": 0,
"chtxtvar204": 0,
"chtxtvar205": 0,
"chtxtvar206": 0,
"chtxtvar207": 0,
"chtxtvar208": 0,
"chtxtvar209": 0,
"chtxtvar210": 0,
"chtxtvar211": 0,
"chtxtvar212": 0,
"chtxtvar213": 0,
"chtxtvar214": 0,
"chtxtvar216": 0,
"chtxtvar218": 0,
"chtxtvar223": 0,
"chtxtvar227": 0,
"chtxtvar229": 0,
"chtxtvar233": 0,
"chtxtvar234": 0,
"chtxtvar235": 0,
"chtxtvar237": 0,
"chtxtvar242": 0,
"chtxtvar243": 0,
"chtxtvar244": 0,
"chtxtvar245": 1,
"chtxtvar246": 0,
"chtxtvar248": 1,
"chtxtvar249": 0,
"chtxtvar250": 0,
"chtxtvar251": 0,
"chtxtvar252": 0,
"chtxtvar253": 0,
"chtxtvar254": 0,
"chtxtvar255": 0,
"chtxtvar256": 0,
"chtxtvar257": 0,
"chtxtvar258": 0,
"chtxtvar259": 0,
"chtxtvar260": 0,
"chtxtvar261": 0,
"chtxtvar262": 0,
"chtxtvar265": 0,
"chtxtvar266": 0,
"chtxtvar267": 0,
"chtxtvar268": 0,
"chtxt14var1": 1,
"chtxt14var3": 0,
"chtxt14var4": 1,
"chtxt14var5": 0,
"chtxt14var6": 0,
"chtxt14var7": 0,
"chtxt14var8": 0,
"chtxt14var9": 0,
"chtxt14var10": 1,
"chtxt14var11": 0,
"chtxt14var12": 0,
"chtxt14var13": 0,
"chtxt14var14": 0,
"chtxt14var15": 0,
"chtxt14var16": 0,
"chtxt14var17": 0,
"chtxt14var18": 0,
"chtxt14var19": 0,
"chtxt14var20": 0,
"chtxt14var21": 0,
"chtxt14var22": 0,
"chtxt14var23": 0,
"chtxt14var25": 0,
"chtxt14var26": 0,
"chtxt14var27": 0,
"chtxt14var29": 0,
"chtxt14var30": 0,
"chtxt14var31": 0,
"chtxt14var32": 0,
"chtxt14var33": 0,
"chtxt14var34": 0,
"chtxt14var35": 0,
"chtxt14var36": 0,
"chtxt14var37": 0,
"chtxt14var38": 0,
"chtxt14var39": 0,
"chtxt14var40": 0,
"chtxt14var41": 0,
"chtxt14var42": 0,
"chtxt14var43": 0,
"chtxt14var44": 0,
"chtxt14var45": 0,
"chtxt14var47": 1,
"chtxt14var48": 0,
"chtxt14var49": 0,
"chtxt14var50": 0,
"chtxt14var51": 0,
"chtxt14var52": 0,
"chtxt14var55": 0,
"chtxt14var56": 0,
"chtxt14var57": 0,
"chtxt14var58": 0,
"chtxt14var59": 0,
"chtxt14var60": 0,
"chtxt14var61": 0,
"chtxt14var62": 0,
"chtxt14var63": 0,
"chtxt14var64": 0,
"chtxt14var65": 0,
"chtxt14var66": 0,
"chtxt14var67": 0,
"chtxt14var68": 0,
"chtxt14var69": 0,
"chtxt14var70": 0,
"chtxt14var71": 0,
"chtxt14var72": 0,
"chtxt14var73": 0,
"chtxt14var74": 0,
"chtxt14var77": 0,
"chtxt14var78": 0,
"chtxt14var80": 0,
"chtxt14var81": 0,
"chtxt14var84": 0,
"chtxt14var85": 0,
"chtxt14var87": 0,
"chtxt14var88": 0,
"chtxt14var89": 0,
"chtxt14var90": 0,
"chtxt14var91": 0,
"chtxt14var92": 0,
"chtxt14var93": 0,
"chtxt14var94": 0,
"chtxt14var95": 0,
"chtxt14var96": 0,
"chtxt14var97": 0,
"chtxt14var98": 0,
"chtxt14var99": 0,
"chtxt14var100": 0,
"chtxt14var101": 0,
"chtxt14var102": 0,
"chtxt14var103": 0,
"chtxt14var105": 0,
"chtxt14var106": 0,
"chtxt14var107": 0,
"chtxt14var108": 0,
"chtxt14var109": 0,
"chtxt14var111": 0,
"chtxt14var113": 0,
"chtxt14var114": 0,
"chtxt14var117": 0,
"chtxt14var118": 0,
"chtxt14var120": 0,
"chtxt14var123": 0,
"chtxt14var124": 0,
"chtxt14var125": 0,
"chtxt14var126": 0,
"chtxt14var128": 0,
"chtxt14var130": 0,
"chtxt14var133": 0,
"chtxt14var135": 0,
"chtxt14var137": 0,
"chtxt14var138": 0,
"chtxt14var140": 0,
"chtxt14var144": 0,
"chtxt14var148": 0,
"chtxt14var150": 0,
"chtxt14var153": 0,
"chtxt14var156": 0,
"chtxt14var158": 0,
"chtxt14var160": 0,
"chtxt14var163": 0,
"chtxt14var169": 0,
"chtxt14var170": 0,
"chtxt14var176": 0,
"chtxt14var181": 0,
"chtxt14var182": 0,
"chtxt14var196": 0,
"chtxt14var197": 0,
"chtxt14var199": 0,
"chtxt14var202": 0,
"chtxt14var205": 0,
"chtxt14var212": 0,
"chtxt14var213": 0,
"chtxt14var229": 0,
"chtxt14var237": 0,
"chtxt14var245": 0,
"chtxt14var246": 0,
"chtxt14var247": 0,
"chtxt14var249": 0,
"chtxt14var250": 0,
"chtxt14var256": 0,
"chcall72var1": 0,
"chcall72var2": 0,
"chcall14var1": 0,
"chcall14var2": 0,
"chbot72var1": 1,
"chbot14var1": 1,
"chmf0var1": 0,
"chmf0var2": 0,
"chmf0var3": 0,
"chmf0var4": 0,
"chmf0var5": 0,
"chmf0var6": 0,
"chmf0var7": 0,
"chmf0var8": 0,
"chmf72var1": 1,
"chmf72var2": 1,
"chmf72var3": 1,
"chmf72var4": 0,
"chmf72var5": 0,
"chmf72var6": 0,
"chmf72var7": 0,
"chmf72var8": 0,
"chmf72var9": 1,
"chmf72var10": 1,
"chmf72var11": 0,
"chmf14var1": 1,
"chmf14var2": 1,
"chmf14var3": 1,
"chmf14var4": 0,
"chmf14var5": 0,
"chmf14var6": 1,
"chmf14var7": 0,
"chmf14var8": 0,
"chmf14var9": 1,
"chmf14var10": 1,
"chmf14var11": 0,
"chret72var1": 0,
"chret14var1": 0,
"chsales72var1": 0,
"chsales14var1": 0,
"chvzupvar1": 0,
"chpayvar1": 1
}
}
}
}

In [5]:
# Sample response from original endpoint

response = {
    "timeTaken": 57,
    "serviceHeader": {
        "clientId": "Streams",
        "clientTransactionId": "<Unique Random Number>",
        "modelName": "channel-hop-model",
        "modelVersion": "v1",
        "custId": "Not Applicable",
        "acctNo": "Not Applicable",
        "mtn": "9999999999",
        "aiProcessorLogID": "5b0932ad-a966-41b0-ba0a-e867d12d371f"
    },
    "serviceResponse": {
        "result": {
            "prediction": [
                "ch_other"
            ],
            "probabilities": {
                "ch_intl": 0.02602,
                "ch_other": 0.8867,
                "ch_stkr_bill_shk_bill_expln": 0.06614,
                "ch_tech_coach": 0.00605,
                "ch_upgr_pred_exp": 0.01509
            },
            "timeTaken": 51.62501335144043
        }
    }
}
